In [7]:
# Choose kernel (your pipenv name as in `pipenv --venv`) 

In [8]:
import os
import pandas as pd
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.oauth2.service_account import Credentials
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from core.apis.google_cloud_api import get_google_cloud_credentials_dict
from core import database
import sys
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

In [9]:
def classify_text(row):
    """
    Classifying Content in a String

    Args:
      text_content The text content to analyze. Must include at least 20 words.
    """

    text_content = row.description
    try:
        if text_content:
            client = language_v1.LanguageServiceClient()

            # Available types: PLAIN_TEXT, HTML
            type_ = enums.Document.Type.PLAIN_TEXT

            # Optional. If not specified, the language is automatically detected.
            # For list of supported languages:
            # https://cloud.google.com/natural-language/docs/languages
            language = "en"
            document = {"content": text_content, "type": type_, "language": language}

            response = client.classify_text(document)
            respjson = MessageToJson(response)
            respdict = MessageToDict(response)
            row.at['g_classn_response'] = respjson
            # Loop through classified categories returned from the API
            highestconfitem = ''
            highestconfscore = 0
            for itemindex in range(len(respdict['categories'])):
                if respdict['categories'][itemindex]['confidence'] >= highestconfscore:
                    highestconfscore = respdict['categories'][itemindex]['confidence']
                    highestconfitem = respdict['categories'][itemindex]['name']

            # https://cloud.google.com/natural-language/docs/categories
            row.at['g_description_class'] = highestconfitem
            row.at['g_desc_cls_confidence'] = highestconfscore

            return row
        return row
    except:
        return row


In [10]:
def tendersQuery():
    query = \
        """
        SELECT            
        distinct t.buyer as src_name,
        t.json -> 'releases' -> 0 -> 'tender' ->> 'description' as description,
        t.source as source,
        t.ocid as ocid,
        t.json::text as ocds_json

          FROM ocds.ocds_tenders_view as t
        WHERE TRUE
          AND t.buyer NOTNULL
          AND source IN (SELECT DISTINCT (scraper) FROM clients.contracts_advance_sources)              
          AND date_created >= '2016-05-23'
          AND date_created <= now()
          AND date_created >= now()::DATE - INTERVAL '1 day'
          AND NOT exists(SELECT 1 FROM ocds.duplicates where t.json -> 'releases' -> 0 ->> 'ocid' = ocid)
          AND t.countryname IN ('United Kingdom','England','Scotland','Northern Ireland','Wales')
        ;"""
    return query


In [11]:
def remove_unicode(text):
    ''' Remove unicode chars '''
    text = text.decode("ascii", errors="ignore").encode()
    return text

In [12]:
# Establish connection to staging database
connection = database.Connection(*database.getDBCredsFromUrl(os.environ['STAGING_DB_URL']))

Connected.


In [13]:
# Construct SQL query
query = tendersQuery()

In [14]:
# Execute SQL query
df = pd.read_sql(query, con=connection.con)

In [15]:
# Resize dataframe for inspection
df = df[:100]

In [16]:
# Run dataframe through Google NL API
df = df.apply(classify_text, axis=1)

In [20]:
df.columns

Index([u'description', u'g_classn_response', u'g_desc_cls_confidence',
       u'g_description_class', u'ocds_json', u'ocid', u'source', u'src_name'],
      dtype='object')

In [36]:
# Extract relevant columns
df1 = df[['ocid','src_name','description','g_description_class','g_desc_cls_confidence']]

In [ ]:
# Format to show all text
pd.set_option('display.max_colwidth', -1)

In [41]:
# Show top X rows
x = 10
df1.head(x)


ocid  \
0  ocds-0c46vo-0133-597846-2019                            
1  ocds-0c46vo-0022-DEC319879                              
2  ocds-0c46vo-0001-489e58e0-2c61-4e28-9a89-b0441a7adb2f   
3  ocds-0c46vo-0133-598214-2019                            
4  ocds-0c46vo-0133-597883-2019                            
5  ocds-0c46vo-0133-595370-2019                            
6  ocds-0c46vo-0133-596954-2019                            
7  ocds-0c46vo-0133-595779-2019                            
8  ocds-0c46vo-0133-598011-2019                            
9  ocds-0c46vo-0007-47144822                               

                                                                           src_name  \
0  Aberdeen City Council                                                              
1  Aberystwyth University                                                             
2  AGH Solutions Ltd                                                                  
3  AGH Solutions Ltd                                                                  
4  Agri-Food Biosciences Institute                                                    
5  Antrim and Newtownabbey Borough Council                                            
6  Antrim and Newtownabbey Borough Council                                            
7  Associated British Ports Ltd                                                       
8  Association of North East Councils Ltd trading as NEPO (Central Purchasing Body)   
9  Balfour Beatty VINCI Systra JV                                                     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 